In [1]:
from collections import Counter
import pandas as pd
import numpy as np 

We downloaded the "AUTO MPG" from the excel file mentioned in the question . Now we can apply changes as asked in the questions . 

* PART A 

For discrete attributes, apply a one-hot encoding, and for non numeric ordinal
attributes, apply integer mapping. Handle outliers/noise and save this in a file.

In this part of the question , we need to apply one-hot encoding to the dataset for the discrete attributes . Now we analyzed the features given in the dataset and get to know that cylinders , model year and  origin . Thus we need to first apply one hot encoding to these . 

In [2]:
dataset = pd.read_csv("Assignment-1 - Auto MPG.csv")

# lets get the unique values of the cylinders first . 

unique_cylinder_values = dataset['cylinders'].unique() ; 

print(unique_cylinder_values) ; 

# now we can add new columns to the dataset for each discrete value and if the entry has that no of cylinders the column corresponding to it will become 1 else 0 ; 

for val in unique_cylinder_values: 
    if (pd.notna(val)) :
        dataset[f'cylinder_{val}'] = 0 
        for i in range(len(dataset)):  
            if dataset.loc[i, 'cylinders'] == val:
                dataset.loc[i, f'cylinder_{val}'] = 1 
  
# thus if number of cylinders for a entry is 4 then cylinder_4 column will be 1 for it otherwise it will be 0 . 

# similiarly we can do for the origin values as follows : 

unique_origin_values = dataset['origin'].unique() ; 

for val in unique_origin_values:
    if (pd.notna(val)):
        dataset[f'origin_{val}'] = 0 
        for i in range(len(dataset)):  
            if dataset.loc[i, 'origin'] == val:
                dataset.loc[i, f'origin_{val}'] = 1  

unique_model_year_values = dataset['model year'].unique() ; 

for val in unique_model_year_values:
    if (pd.notna(val)):
        dataset[f'model year_{val}'] = 0 
        for i in range(len(dataset)):  
            if dataset.loc[i, 'model year'] == val:
                dataset.loc[i, f'model year_{val}'] = 1  
                
dataset.drop(['cylinders', 'origin' , 'model year'], axis=1, inplace=True)  ; 

# thus we changed the discrete attributes with their one-hot encoding . 
# Now we need to apply integer mapping for non - numeric ordinal attributes . 

# We dont have any such attribute so we can skip this step . Check csv file for the same . 




[8 4 6 3 5]


Now there are 0 non - numeric ordinal attributes . Thus we can skip the step of applying integer mapping to such values . Now we need to handle outliers/noise and save this in a file . 

In [3]:
def calculate_mean(data) : 
  if (len(data) == 0):
    return None ; 
  
  return sum(data) / len(data) ; 

def calculate_median(data):
    sorted_data = sorted(data)
    n = len(sorted_data)
    
    if n == 0:
        return None
    
    if n % 2 == 1:
        return sorted_data[n // 2]
    middle1 = sorted_data[n // 2 - 1]
    middle2 = sorted_data[n // 2]
    return (middle1 + middle2) / 2
  
def calculate_mode(data):
    if len(data) == 0:
        return None
    counter = Counter(data)
    mode_data = counter.most_common(1)
    return mode_data[0][0] if mode_data else None

dataset['horsepower'] = pd.to_numeric(dataset['horsepower'] , errors='coerce') ; 

numerical_cols = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration']

# below is the implementation for my numerical value
def fill_missing_numerical(dataset, column):
  mean_val = calculate_mean(dataset[column].dropna()) ; 
  median_val = calculate_median(dataset[column].dropna()) ; 
  
  if (dataset[column].skew() > 0.5 or dataset[column].skew() < 0.5):
    
    dataset[column].fillna(median_val , inplace = True) ; 
    
  else:
    dataset[column].fillna(mean_val , inplace = True) ; 

# below is the implementation for my one-hot encoded/categorical value
def fill_missing_categorical(dataset , column): 
  mode_val = calculate_mode(dataset[column].dropna()) ;
  
  dataset[column].fillna(mode_val , inplace = True) ; 
  
  
# i am using IQR as taught in class
def remove_outliers(dataset ,  column):
  q1 = dataset[column].quantile(0.25) ; 
  
  q3 = dataset[column].quantile(0.75) ; 
  
  iqr = q3 - q1 ; 
  
  lower_bound = q1 - 1.5 * iqr ; 
  upper_bound = q3 + 1.5 * iqr ; 
  
  
  dataset = dataset[(dataset[column] >= lower_bound) & (dataset[column] <= upper_bound) ] ; 
  
  
  return dataset ; 
def clean_data(dataset , numerical_cols):
  for column in dataset.columns:
    if (column in numerical_cols):
      fill_missing_numerical(dataset , column ) ; 
      
      dataset = remove_outliers(dataset , column ) ; 
    
    else: 
      fill_missing_categorical(dataset , column) ;
      
  
  return dataset ; 


dataset_new = clean_data(dataset , numerical_cols) ; 


      
dataset_new.to_csv('Cleaned_Dataset.csv' , index=False) ; 
      
      
    

C:\Users\Aditya\AppData\Local\Temp\ipykernel_29628\1747376723.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(median_val , inplace = True) ;
C:\Users\Aditya\AppData\Local\Temp\ipykernel_29628\1747376723.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

# Part b 


In this part of the question , we need to calculate the mean x bar and variance sigma^2 of the data in it . 

In this we have datapoints with d number of features in them . Thus the mean x bar will be of the form d*1 , where d is the dimension(number of features of each datapoint .) 

In [4]:
dataset_cleaned = pd.read_csv("Cleaned_Dataset.csv") ; 

# now the dataset_cleaned contains the datapoints with the one-hot encoded features (thus there are now 14 columns for each datapoint) . 
# the mean is as follows : 

# converting the dataframe into numpy array for easier manupilation and calculation . 

dataset_cleaned = dataset_cleaned.to_numpy() ; 

#print(dataset)

# Now lets calculate the mean of the values . 

no_of_datapoint , no_of_features = dataset_cleaned.shape ; 


mean_x = np.zeros(no_of_features) ; # as the array of x bar will be of the shape d * 1 ; 

# print(dataset_cleaned)
for i in range(no_of_datapoint):
  
  
  mean_x += dataset_cleaned[i] ; # this way i am adding the ith dimensional feature to the ith index in the mean_x array . 




mean_x /= no_of_datapoint ; 

print(mean_x) 

   


[2.36685185e+01 1.87691799e+02 1.01492063e+02 2.93610714e+03
 1.56063492e+01 2.38095238e-01 5.21164021e-01 2.22222222e-01
 1.05820106e-02 7.93650794e-03 6.19047619e-01 2.06349206e-01
 1.71957672e-01 5.29100529e-02 7.14285714e-02 6.87830688e-02
 9.78835979e-02 7.14285714e-02 7.93650794e-02 8.73015873e-02
 7.40740741e-02 9.52380952e-02 7.14285714e-02 6.87830688e-02
 2.64550265e-03 7.67195767e-02 7.93650794e-02]


Now we will calculate the variance of the dataset using the features and the transpose formulae given in the question . 

In [5]:
# now we have dataset_cleaned as a numpy array so we can calculate the variance as follows : 

# mean_X is the mean of X ie X bar 

variance = 0 ; 

for i in range(no_of_datapoint) : 
  
  
  first_term = (dataset_cleaned[i] - mean_x) ; 

  second_term = (dataset_cleaned[i] - mean_x) ; 

  variance += np.dot(first_term , second_term) ; 
  
variance /= no_of_datapoint ; 


print(variance)

# Thus we get the variance(sigma ^ 2) from this . 

684779.143322625


# Part C

In this part of the question , we need to first see that the variance of the data is highly dominated by few features compared to other features . For this lets first print the individuals terms added by these features . ie. (xi - x_bar).T * (xi - x_bar) . 

In [6]:
# Lets first find the contribution of the term (xi - x_bar).T * (xi - x_bar) from each feature . It is as follows : 

squared_diffs = np.zeros(no_of_features) ; 

for i in range(no_of_datapoint) : 
  diff = dataset_cleaned[i] - mean_x ; 
  
  squared_diffs += diff ** 2 ; 
  

variance = squared_diffs /no_of_datapoint ; 


print(variance) ; # now we are printing the variance array to check which term is more dominating , the feature which is more dominating . 


dominant_features = np.argsort(-variance) ;  

print(f"\nFeatures contributing the most variance (in descending order) :  {dominant_features}") ; 

# Thus we can see that features are contributing differently to the variance . 


[5.63662840e+01 9.59135541e+03 1.08743512e+03 6.74035926e+05
 5.97715545e+00 1.81405896e-01 2.49552084e-01 1.72839506e-01
 1.04700316e-02 7.87351978e-03 2.35827664e-01 1.63769211e-01
 1.42388231e-01 5.01105792e-02 6.63265306e-02 6.40519582e-02
 8.83023991e-02 6.63265306e-02 7.30662635e-02 7.96800202e-02
 6.85871056e-02 8.61678005e-02 6.63265306e-02 6.40519582e-02
 2.63850396e-03 7.08336833e-02 7.30662635e-02]

Features contributing the most variance (in descending order) :  [ 3  1  2  0  4  6 10  5  7 11 12 16 21 19 26 18 25 20 22 17 14 23 15 13
  8  9 24]


Now lets normalize each feature with its mean and variance and then compute the variance normalized data . We use the formulae 

$$
X_{normalised} = \frac{X_{col} - \mu_{col}}{\sigma_{col}}
$$

In [7]:
# we have mean for each feature in mean_X and variance of each feature in variance , thus we can proceed . 

normalized_data = np.zeros_like(dataset_cleaned) ; 

for i in range(no_of_datapoint): 
  normalized_data[i] = (dataset_cleaned[i] - mean_x) / (np.sqrt(variance)) ; 

# thus we normalized each datapoint in this 

normalized_variance = 0 ; 

for i in range(no_of_datapoint) : 
  
  
  normalized_variance += np.dot(normalized_data[i] , normalized_data[i]) ; 
  
normalized_variance /= no_of_datapoint ; 


print(normalized_variance)

27.000000000000014


Thus we get variance as 27 as there are 27 number of columns and each column contributed equally to the variance . 

# Part D

In this part of the question , we need to check in the given dataset with a 5\% level of significance , test if the number model year has any effect on the number of cylinders . We are also given reference to table for the same . Thus we are using cleaned data which no missing values and outliers . 

We are using cleaned dataset for this purpose . 

We will be using Chi-Squared test for finding the dependency of number of cylinders on model year . The hypothesis are as follows : 

## Hypotheses for Chi-Squared Test

### Null Hypothesis (H0)
The null hypothesis states that there is no association between the number of cylinders and the model year of the cars. In other words, the two variables are independent.

**H₀**: The number of cylinders is independent of the model year.

### Alternative Hypothesis (H1)
The alternative hypothesis states that there is an association between the number of cylinders and the model year of the cars. This means that the two variables are dependent.

**H₁**: The number of cylinders is dependent on the model year.


In [8]:
column_names = [
    'mpg', 'displacement', 'horsepower', 'weight', 'acceleration',
    'cylinder_8', 'cylinder_4', 'cylinder_6', 'cylinder_3', 'cylinder_5',
    'origin_1.0', 'origin_3.0', 'origin_2.0',
    'model year_70.0', 'model year_71.0', 'model year_72.0',
    'model year_73.0', 'model year_74.0', 'model year_75.0',
    'model year_76.0', 'model year_77.0', 'model year_78.0',
    'model year_79.0', 'model year_80.0', 'model year_99999.0',
    'model year_81.0', 'model year_82.0'
]

dataset_chi = pd.DataFrame(dataset_cleaned , columns=column_names) ; 



cylinder_cols = dataset_chi.filter(like='cylinder_')
model_year_cols = dataset_chi.filter(like='model year_')


def get_max_column(df):
    return df.idxmax(axis=1)

cylinder_max = get_max_column(cylinder_cols)
model_year_max = get_max_column(model_year_cols)


contingency_table = pd.crosstab(cylinder_max, model_year_max)


print("Contingency Table:")
print(contingency_table)

Contingency Table:
col_0       model year_70.0  model year_71.0  model year_72.0  \
row_0                                                           
cylinder_3                0                0                1   
cylinder_4                7               12               13   
cylinder_5                0                0                0   
cylinder_6                4                8                0   
cylinder_8               10                7               12   

col_0       model year_73.0  model year_74.0  model year_75.0  \
row_0                                                           
cylinder_3                1                0                0   
cylinder_4               11               15               12   
cylinder_5                0                0                0   
cylinder_6                8                7               12   
cylinder_8               17                5                6   

col_0       model year_76.0  model year_77.0  model year_78.0  \
row_

### Formula

The Chi-Squared statistic is calculated using the following formula:

$$
\chi^2 = \sum \frac{(O_i - E_i)^2}{E_i}
$$

Where:
- \( \chi^2 \) = Chi-Squared statistic
- \( O_i \) = Observed frequency for each category
- \( E_i \) = Expected frequency for each category, calculated as:

$$
E_i = \frac{(Row \, Total) \times (Column \, Total)}{Grand \, Total}
$$

Now we will be using Chi-test to test if the number model year has any effect on the number of cylinders . 

In [9]:
# We got the contigency table in the above step . 
observed_data = contingency_table.values ; 

# print(observed_data)
row_sums = observed_data.sum(axis=1) ; 

column_sums = observed_data.sum(axis=0) ; 

total_sum = observed_data.sum() ; 

# print(row_sums) ; 

expected = np.outer(row_sums , column_sums ) / (total_sum) ; 

chi_squared_val = ((observed_data - expected) ** 2 / expected).sum() ; 

degree_of_freedom = (len(contingency_table.index) - 1) * (len(contingency_table.columns) - 1) ; 

print(chi_squared_val) ; 

print(degree_of_freedom)


101.55958345492391
52


Thus we get the chi_squared_value($\chi^2$) as 101.56 . Now we know that the degree of freedom(dof) is (r-1) * (c-1) = 52 in this case as r = 5 and c = 14 . Now we know significance as 5 % . Thus it is a one sided test so the level of significance will remain as 5 % . Thus we can look in the table and answer the hypothesis . We can see that the threshold in the table is 73.810 . Thus ($\chi^2$) > 69.832 . Thus we reject the null hypothesis that they are independent and thus conclude that number of cylinders are not independent of model years . 